# **Installations And Pretrained Models File Fetch**

Installing libraries and important files

In [ ]:
!pip install -U flask-cors
!pip install flask-ngrok
!pip install pyngrok
!pip install -U sentence_transformers
!pip install keras-preprocessing
!pip install sk-video
!wget -O 'c3d.py' 'https://drive.google.com/uc?id=1RC9trsgIo2OsM8dV7CPg73drqSRZMr8j&confirm=t'
!wget -O 'sports1M_utils.py' 'https://drive.google.com/uc?id=1S49OSW2pCUPPD9F-763mG-aBx2gBOr2i&confirm=t'

# **Recommendations And Search Cell**

In [ ]:
#recommender cell
from flask import Flask
from flask_ngrok import run_with_ngrok
import numpy as np
from collections import defaultdict
import tqdm.notebook as tqdm #for loader graphic
import networkx as nx
# import torch
from sentence_transformers import SentenceTransformer, util
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
import re
import collections
import nltk
import numpy as mp
import pandas as pd
from flask import request
from flask import jsonify
df = pd.read_csv('/kaggle/input/datacaptions/data.csv')
df = df.groupby('VideoID', as_index=False).first()


# len(df)


#data cleaning of the descriptions
nltk.download('punkt')
# import requests
# import bs4
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')


df.head()


#creating instance of sentence transformer
model_transformer = SentenceTransformer('paraphrase-MiniLM-L6-v2')
X = model_transformer.encode(df.Description.values)

def generate_results(test_sentence, status=1):
    print('QUERIED DESC : ',test_sentence)
    data = {}
    Ids = []
    
    #encoding query vector
    query_vec = model_transformer.encode(test_sentence)

    #finding cosine similarity with vectors of database
    results = sklearn.metrics.pairwise.cosine_similarity(
        X, [query_vec], dense_output=True)

    res = []
    for i in range(len(results)):
        res.append([results[i], i])
   #sorting 
    res.sort(reverse=True)
    if status == 0:
        return res
    # print('Recommended Video--------Recommendation')
    
    #returning top30 information retrieval results
    for i in res[:30]:
        data[df.iloc[i[1], 0]] = df.iloc[i[1], 1]
        Ids.append(df.iloc[i[1], 0])
        # print(df.iloc[i[1], 0], "--------", df.iloc[i[1], 1])
    return data,Ids


# **Video Description Generator**

In [ ]:
# -*- coding: utf-8 -*-


from tensorflow import keras
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from tensorflow.keras import backend as K1
from keras.models import *
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3
import os
import cv2
import random
import joblib
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import skvideo.io
import importlib as imp
from c3d import C3D
from keras.models import Model
import sports1M_utils

"""# ***Video PreProcessing***"""

K = 28


#function to fetch equally spaced k frames
def extract_frames_equally_spaced(frames, K):
    n_frames = len(frames)

    splits = np.array_split(range(n_frames), K)
    idx_taken = [s[0] for s in splits]
    sub_frames = []

    for idx in idx_taken:
        sub_frames.append(frames[idx])
    return sub_frames

#padding frames to make net frames same for all
def pad_frames(frames, limit, jpegs=False):
    last_frame = frames[-1]
    if jpegs:
        frames_padded = frames + [last_frame]*(limit-len(frames))
    else:
        padding = np.asarray([last_frame * 0.]*(limit-len(frames)))
        frames_padded = np.concatenate([frames, padding], axis=0)
    return frames_padded

#resizing of the frames
def resize_frames(frames):
    new_frames = []
    for frame in frames:
        new_frame = cv2.resize(frame, (224, 224))
        new_frames.append(new_frame)

    return new_frames

#extracting k frames from the videos
def video_to_frames(input_loc, K):
    cap = cv2.VideoCapture(input_loc)

    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        frames.append(frame)
        count = count + 1

        if (count > (video_length-1)):
            cap.release()
            break
    if len(frames) < K:
        frames = pad_frames(frames, K, True)
    else:
        frames = extract_frames_equally_spaced(frames, K)

    return frames


#model for global feature extraction
def get_model():
    i = tf.keras.layers.Input([224, 224, 3], dtype=tf.float32)
    x = tf.keras.applications.inception_v3.preprocess_input(i)
    model = InceptionV3(weights="imagenet", include_top=False,
                        input_shape=(224, 224, 3), pooling='avg')
    out = model(x)
    model = tf.keras.models.Model(inputs=[i], outputs=out)
    return model
    # model.summary()



google_net_model = get_model()

#extracting global-features from the frames
def extract_global_features(save, path):
    if not os.path.exists('global_features'):
        os.makedirs('global_features')

    frames = video_to_frames(path, K)
    frames = resize_frames(frames)
    frames = np.array(frames)

    feature_vector = google_net_model.predict(frames, batch_size=128)
    feature_path = 'global_features/'+'global'+'.npy'
    np.save(feature_path, feature_vector)
    return feature_vector



#extracting motion features from the frames
def extract_frames_equally_spaced_motion(frames, K):
    n_frames = len(frames)
    if (n_frames < K):
        while len(frames) < K:
            frames.append(frames[-1])
        return frames

    # print('Nframes : ',n_frames)
    splits = np.array_split(range(n_frames), K)
    idx_taken = [s[0] for s in splits]
    sub_frames = []

    for idx in idx_taken:
        sub_frames.append(frames[idx])
    return sub_frames


#creating instance of pretrained model
base_model = C3D(weights='sports1M')
model_motion = Model(inputs=base_model.input,
                  outputs=base_model.get_layer('fc6').output)


# extracting motion features over windows using previously defined functions
def extract_motion_features(save, path):
    imp.reload(sports1M_utils)
    if not os.path.exists('motion_features1'):
        os.makedirs('motion_features1')


    
    frames = video_to_frames(path, K)
    frames = resize_frames(frames)
    frames = np.array(frames)

    windows = []
    org_vid = frames
    vid = []
    motion_features_video = []
    for i in range(len(frames)):
        vid.append(frames[i])
        if i == 15:
            windows.append(vid)
        elif i >= 16:
            vid.pop(0)
            windows.append(vid)
    print(len(windows))
    print(len(windows[0]))

    windows = extract_frames_equally_spaced_motion(windows, 28)
    for window in windows:
        x = sports1M_utils.preprocess_input(np.array(window))
        feature_vector = model_motion.predict(x)
        feature_vector = feature_vector[0]
        feature_vector = np.array(feature_vector)

        motion_features_video.append(feature_vector)
    motion_features_video = np.array(motion_features_video)

    path = 'motion_features1/'+'motion'+'.npy'
    np.save(path, motion_features_video)
    return motion_features_video



#attention mechanism class used to create model skeltal and load the weights obtained from the training of model
class attention(tf.keras.layers.Layer):

    def __init__(self, return_sequences=True, **kwargs):
        self.return_sequences = return_sequences
        super(attention, self).__init__()

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'return_sequences': self.return_sequences
        })
        return config

    def build(self, input_shape):

        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(attention, self).build(input_shape)

    def call(self, x):

        e = K1.tanh(K1.dot(x, self.W)+self.b)
        a = K1.softmax(e, axis=1)
        output = x*a

        if self.return_sequences:
            return output

        return K1.sum(output, axis=1)

#loading models (encoder and decoder) and fetching descriptions using the greedy search
class Video2Text(object):
    ''' Initialize the parameters for the model '''

    def __init__(self):
        self.latent_dim = 512
        self.num_encoder_tokens = 2048
        self.num_decoder_tokens = 1500
        self.time_steps_encoder = 28
        self.time_steps_decoder = None
        self.max_probability = -1

        # processed data
        self.encoder_input_data = []
        self.decoder_input_data = []
        self.decoder_target_data = []
        self.tokenizer = None

        # models
        self.encoder_model = None
        self.decoder_model = None
        self.inf_encoder_model = None
        self.inf_decoder_model = None
        self.save_model_path = '/kaggle/input/mp-model-weights'
        self.test_path_global = 'global_features'
        self.test_path_motion = 'motion_features1'
        self.test_path_local = 'local_features1'

    def load_inference_models(self):
        # load tokenizer
        with open(os.path.join(self.save_model_path, 'tokenizer' + str(self.num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)
            print('tokeniser loaded')

        # inference encoder model
        self.inf_encoder_model = tf.keras.models.load_model(os.path.join(
            self.save_model_path, 'encoder_model.h5'), custom_objects={'attention': attention})
        print('encoder loaded')

        # inference decoder model
        decoder_inputs = tf.keras.layers.Input(
            shape=(None, self.num_decoder_tokens))
        decoder_dense = tf.keras.layers.Dense(
            self.num_decoder_tokens, activation='softmax')

        decoder_gru = tf.keras.layers.GRU(
            self.latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = tf.keras.layers.Input(shape=(self.latent_dim,))
        # decoder_state_input_c = tf.keras.layers.Input(shape=(self.latent_dim,))
        decoder_states_inputs = [decoder_state_input_h]
        decoder_outputs, state_h = decoder_gru(
            decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = tf.keras.models.Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)

        self.inf_decoder_model.load_weights(os.path.join(
            self.save_model_path, 'decoder_model_weights.h5'))
        print('Loaded Inference Model')

    def index_to_word(self):
        # inverts word tokenizer
        index_to_word = {value: key for key,
                         value in self.tokenizer.word_index.items()}
        return index_to_word

    def greedy_search(self, f):
        """
        :param f: the loaded numpy array after creating videos to frames and extracting features
        :return: the final sentence which has been predicted greedily
        """
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(f.reshape(-1, 28, 10240))
        states_value = np.array(states_value).reshape((1, 512))
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h = self.inf_decoder_model.predict(
                [target_seq] + [states_value])
            states_value = [h]
            output_tokens = output_tokens.reshape(self.num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def test(self, global_features, motion_features):
        X_test = np.concatenate(
            (global_features, motion_features, motion_features), axis=1)
        x = X_test.reshape(-1, 28, 10240)
        print(x.shape)
        decoded_sentence = self.greedy_search(x)
        return (decoded_sentence)

    
#object of above class    
c = Video2Text()
c.load_inference_models()



#main driver function for generating the descriptions
def get_description(path):

    frames = resize_frames(video_to_frames(path, K))

    print(len(frames))
    print(frames[0].shape)

    """# ***Features Extraction***

  # ***Global Features***
  """

    # Extract global features for training

    global_features = extract_global_features(True, path)

    print(len(global_features))
    print(global_features[0].shape)

    """# ***Motion Features***"""

    # Extracting Motion Features

    motion_features = extract_motion_features(True, path)

    print(motion_features.shape)

    """# ***GetDescription***"""


    return (c.test(global_features, motion_features))


# print(get_description('content/_0nX-El-ySo_83_93.avi'))
# print(get_description(
#     '/kaggle/working/_0nX-El-ySo_83_93.mp4'))


# **Flask Server**

In [ ]:
#installing local tunnel to expose local server to the outer world
!pip install flask-localtunnel


In [ ]:
from flask import Flask
from flask import send_from_directory
from flask_ngrok import run_with_ngrok
from distutils.log import debug
from fileinput import filename
from flask import *  
import datetime
import math
import random
from pyngrok import ngrok
from flask_cors import CORS
from flask_lt import run_with_lt#
df1 = pd.read_csv('/kaggle/input/datacaptions/data.csv')


#use this in case of using ngrok instead of local tunnel
ngrok.set_auth_token("2N1pQ8aidKgB9HZeGmFVyvI1wFZ_qB94CGiuXoAE9q5vASKP")


#Storage
#here for simplicity one user is considered otherwise dictionary of dictionaries can be created for multiple users,
#where first dictionary will contain userID as key
uploads_with_desc = {} #storing total uploaded data urls for particular user

uploads = [] #contains file_name
uploads_weight = [] #contains weight



seed_weight = 0.1 #will increase it gradually to give more weight to recent videos


#map to check for case of reupload
check_reupload = defaultdict(int)

#initial time
initial_time = datetime.datetime(2023, 3, 14, 12, 0, 0)

#App

app = Flask(__name__)
CORS(app)  #adding support for cors by sending cors allow headers and bypassing tunnel reminder in response by the server
app.config['CORS_ALLOW_HEADERS'] = ['Content-Type', 'Authorization', 'Bypass-Tunnel-Reminder','bypass-tunnel-reminder']

run_with_lt(app) #running with local tunnel
# run_with_ngrok(app) 

# @app.after_request
# def add_cors_headers(response):
#     response.headers['Access-Control-Allow-Origin'] = '*'
#     return response


#file upload
@app.route('/upload', methods = ['POST'])  
def file_upload():  
    if request.method == 'POST':  
        f = request.files['file']
        f.save(f.filename)  
        print(f.filename)
        desc = get_description('/kaggle/working/'+f.filename) 
        print(desc)
        if check_reupload[desc] == 0:
            current_time = datetime.datetime.now()
            time_difference = current_time - initial_time
            hours_difference = time_difference.total_seconds() / 3600
            if int(math.ceil(hours_difference))%24 == 0:
                seed_weight+=0.1
            uploads_with_desc[f.filename] = desc
            data,Ids = generate_results(desc)
            
            uploads.extend(data)
            print('Current Upload Len : ',len(Ids),' ',len(uploads))
            temp_weight = []
            for i in range(len(Ids)):
                temp_weight.append(seed_weight)
#             temp_weight = [seed_weight]* 30
#             print('Temp Weight : ',temp_weight)
            uploads_weight.extend(temp_weight)
            check_reupload[desc] = 1
        return render_template("Acknowledgement.html", name = f.filename)  

#serving static files
@app.route('/file/<path:path>', methods=['GET'])
def send_data(path):
    print(path)
    return send_from_directory('/kaggle/input/avi-2-mp4','mp4/'+path+'.mp4')

# fetching results for users search
@app.route('/search/<name>', methods=['GET'])
def search_video(name):
    if (request.method == 'GET'):
        print(name)
        data,_ = generate_results(name)
#         print(data.keys())
        return jsonify(data)



#serving uploaded videos
@app.route('/serveUploadedFile/<path:path>', methods=['GET'])
def send_data_ofUpload(path):
    print(path)
    return send_from_directory('/kaggle/working/',path)

#sending Ids and description of uploaded videos
@app.route('/uploaded/', methods=['GET'])
def uploaded_video_feed_data():
    return (jsonify(uploads_with_desc))



#Returning video for recommendations feed

@app.route('/rec/<name>', methods=['GET'])
def send_rec_data(name):
    print('rec feed path : ',name)
    return send_from_directory('/kaggle/input/avi-2-mp4','mp4/'+ name +'.mp4')
    

@app.route('/rec_dic/', methods=['GET'])
def generate_video_feed_data():
#     print(uploads)
    desc={}
    for id in uploads:
        try:
#              print(df[df['VideoID']==str(id)]['VideoID'].iloc[[0]].values[0])
#              print(df[df['VideoID']==str(id)]['Description'].iloc[[0]].values[0])
             desc[df1[df1['VideoID']==str(id)]['VideoID'].iloc[[0]].values[0] ] = df1[df1['VideoID']==str(id)]['Description'].iloc[[0]].values[0]
        except:
            print('Not Found For ID : ',id)
#         desc[df.iloc[id, 0]] = df.iloc[id, 1]
    print(desc)
    return (jsonify(desc))



if __name__ == '__main__':
    app.run()

***Add Garbage Collector For RAM Optimisations***